In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Concatenate, Add, GlobalAveragePooling1D, GlobalMaxPooling1D, MultiHeadAttention, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention

In [7]:
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout, BatchNormalization, Concatenate, Add, GlobalAveragePooling1D, GlobalMaxPooling1D, MultiHeadAttention, Conv1D
from tensorflow.keras.models import Model

def advanced_weather_model(input_shape, output_shape):
    inputs = Input(shape=(None, input_shape[1]))
    
    x1 = Bidirectional(LSTM(256, return_sequences=True))(inputs)
    x1 = BatchNormalization()(x1)
    x1 = Dropout(0.2)(x1)
    
    # Parallel branch with different kernel sizes
    conv_branch1 = Conv1D(128, kernel_size=3, padding='same', activation='relu')(x1)
    conv_branch2 = Conv1D(128, kernel_size=5, padding='same', activation='relu')(x1)
    conv_branch3 = Conv1D(128, kernel_size=7, padding='same', activation='relu')(x1)
    x = Concatenate()([conv_branch1, conv_branch2, conv_branch3])
    
    x = Dense(512)(x)
    
    # Second block - Deep LSTM stack with residual connections
    lstm_1 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x = Add()([x, lstm_1])
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    lstm_2 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x = Add()([x, lstm_2])
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    attention1 = MultiHeadAttention(num_heads=8, key_dim=32)(x, x, x)
    attention2 = MultiHeadAttention(num_heads=8, key_dim=32)(x, x, x)
    attention_concat = Concatenate()([attention1, attention2])
    
    attention_concat = Dense(512)(attention_concat)
    x = Add()([x, attention_concat])
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Bidirectional(LSTM(512, return_sequences=True))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    
    branch1 = LSTM(256, return_sequences=True)(x)
    branch2 = LSTM(256, return_sequences=True)(x)
    x = Concatenate()([branch1, branch2])

    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = Concatenate()([avg_pool, max_pool])
    
    dense1 = Dense(512, activation='relu')(x)
    dense1 = BatchNormalization()(dense1)
    dense1 = Dropout(0.3)(dense1)
    
    dense2 = Dense(512, activation='relu')(dense1)
    dense2 = BatchNormalization()(dense2)
    dense2 = Dropout(0.3)(dense2)
    dense2 = Add()([dense1, dense2])
    
    dense3 = Dense(256, activation='relu')(dense2)
    dense3 = BatchNormalization()(dense3)
    dense3 = Dropout(0.2)(dense3)

    outputs = Dense(output_shape, activation='linear')(dense3)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [9]:
input_time_steps = 30
num_features = 4
output_days = 5
input_shape = (input_time_steps, num_features)#throwing error for invalid data type so did like this

model = advanced_weather_model(input_shape, output_days)

In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, None, 4)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional (Bidirectional) │ (None, None, 512)         │         534,528 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, None, 512)         │           2,048 │ bidirectional[0][0]        │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, None, 512)         │               0 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d (Conv1D)               │ (None, None, 128)         │         196,736 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_1 (Conv1D)             │ (None, None, 128)         │         327,808 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_2 (Conv1D)             │ (None, None, 128)         │         458,880 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, None, 384)         │               0 │ conv1d[0][0],              │
│                               │                           │                 │ conv1d_1[0][0],            │
│                               │                           │                 │ conv1d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, None, 512)         │         197,120 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_1               │ (None, None, 512)         │       1,574,912 │ dense[0][0]                │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, None, 512)         │               0 │ dense[0][0],               │
│                               │                           │                 │ bidirectional_1[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, None, 512)         │           2,048 │ add[0][0]                  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, None, 512)         │               0 │ batch_normalization_1[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_2               │ (None, None, 512)         │       1,574,912 │ dropout_1[0][0]            │
│ (Bidirectional)               │                           │               

 Total params: 14,200,197 (54.17 MB)

 Trainable params: 14,191,493 (54.14 MB)

 Non-trainable params: 8,704 (34.00 KB)

In [13]:
def load_3d_array_from_excel(filename):
    xls = pd.ExcelFile(filename)
    sheets = xls.sheet_names
    data = []
    for sheet in sheets:
        df = pd.read_excel(xls, sheet_name=sheet)
        data.append(df.values)
    return np.array(data)

In [15]:
X_train = load_3d_array_from_excel('X_train.xlsx')
X_test = load_3d_array_from_excel('X_test.xlsx')
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (1, 1567, 4)
X_test shape: (1, 392, 4)


In [17]:
y_train = X_train[:, 1:, :]  # Use the next 'output_days' days as targets
y_test = X_test[:, 1:, :]
input_time_steps, num_features = X_train.shape[1], X_train.shape[2]#Adjust input and output shape
output_days = 5
output_shape = output_days * num_features

In [19]:
y_train = y_train[:, :output_days, :].reshape(y_train.shape[0], -1)
y_test = y_test[:, :output_days, :].reshape(y_test.shape[0], -1)

In [21]:
#create a model instance
input_shape = (input_time_steps, num_features)
model = advanced_weather_model(input_shape, output_shape)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, None, 4)           │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_4               │ (None, None, 512)         │         534,528 │ input_layer_1[0][0]        │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_8         │ (None, None, 512)         │           2,048 │ bidirectional_4[0][0]      │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_10 (Dropout)          │ (None, None, 512)         │               0 │ batch_normalization_8[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_3 (Conv1D)             │ (None, None, 128)         │         196,736 │ dropout_10[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_4 (Conv1D)             │ (None, None, 128)         │         327,808 │ dropout_10[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1d_5 (Conv1D)             │ (None, None, 128)         │         458,880 │ dropout_10[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_4 (Concatenate)   │ (None, None, 384)         │               0 │ conv1d_3[0][0],            │
│                               │                           │                 │ conv1d_4[0][0],            │
│                               │                           │                 │ conv1d_5[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_6 (Dense)               │ (None, None, 512)         │         197,120 │ concatenate_4[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_5               │ (None, None, 512)         │       1,574,912 │ dense_6[0][0]              │
│ (Bidirectional)               │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_4 (Add)                   │ (None, None, 512)         │               0 │ dense_6[0][0],             │
│                               │                           │                 │ bidirectional_5[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_9         │ (None, None, 512)         │           2,048 │ add_4[0][0]                │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_11 (Dropout)          │ (None, None, 512)         │               0 │ batch_normalization_9[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ bidirectional_6               │ (None, None, 512)         │       1,574,91

 Total params: 14,204,052 (54.18 MB)

 Trainable params: 14,195,348 (54.15 MB)

 Non-trainable params: 8,704 (34.00 KB)

In [23]:
#train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=1,
    verbose=1
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 61s 61s/step - loss: 0.1832 - mae: 0.3051
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - loss: 0.1799 - mae: 0.3024
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - loss: 0.1762 - mae: 0.2992
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step - loss: 0.1726 - mae: 0.2959
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 31s 31s/step - loss: 0.1686 - mae: 0.2916
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - loss: 0.1663 - mae: 0.2886
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - loss: 0.1605 - mae: 0.2828
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.1556 - mae: 0.2790
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.1496 - mae: 0.2726
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.1454 - mae: 0.2674
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.1431 - mae: 0.2629
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 33s 33s/step - loss: 0.1366 - mae: 0.2580
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - loss: 0.1

In [25]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test loss: {test_loss}")
print(f"Test MAE: {test_mae}")
predictions = model.predict(X_test)
predictions_reshaped = predictions.reshape(-1, output_days, num_features)
print("Predictions shape:", predictions_reshaped.shape)

model.save('advanced_weather_prediction_model.h5')
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.tight_layout()
plt.savefig('model_training_history.png')
plt.close()

Test loss: 0.026969540864229202
Test MAE: 0.11695700883865356
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Predictions shape: (1, 5, 4)


In [27]:
num_features = 4
output_days = 5
last_sequence = X_test[-1:] 
next_5_days_prediction = model.predict(last_sequence)
print(f"Prediction shape: {next_5_days_prediction.shape}")
if len(next_5_days_prediction.shape) == 2:
    predictions_reshaped = next_5_days_prediction.reshape(output_days, num_features)
elif len(next_5_days_prediction.shape) == 3:
    # If it's a 3D array (e.g., (batch_size, timesteps, num_features))
    print("The model predicted a 3D array.")
    predictions_reshaped = next_5_days_prediction.reshape(output_days, num_features)
else:
    raise ValueError(f"Unexpected prediction shape: {next_5_days_prediction.shape}")

print("Predictions for the next 5 days:")
print(predictions_reshaped)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction shape: (1, 20)
Predictions for the next 5 days:
[[ 0.7115944   0.60292727  0.06068463  0.00622312]
 [ 0.6178838   0.76891327  0.00751295  0.11754079]
 [ 0.60725033  0.81878245  0.07373429  0.00667943]
 [ 0.5987944   0.67720985  0.05687155 -0.00761911]
 [ 0.2378232   0.6128792   0.01574285  0.00370278]]


In [29]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def create_weather_scaler():
    # Create training data array with min and max values for each feature
    train_data = np.array([
        # Temperature, Humidity, Wind Speed, Precipitation
        [80.0, 65.0, 5.0, 0.0],    # Minimum values
        [90.0, 90.0, 15.0, 25.0],  # Maximum values
        [85.0, 75.0, 10.0, 12.5],  # Middle values to ensure proper scaling
    ])
    scaler = MinMaxScaler()
    scaler.fit(train_data)
    
    return scaler

def inverse_transform_weather_predictions(scaled_predictions):
    if len(scaled_predictions.shape) != 2:
        scaled_predictions = scaled_predictions.reshape(-1, 4)
    scaled_predictions = np.clip(scaled_predictions, 0, 1)
    scaler = create_weather_scaler()
    original_scale = scaler.inverse_transform(scaled_predictions)
    predictions_dict = {
        'Temperature_F': original_scale[:, 0],
        'Relative_Humidity': original_scale[:, 1],
        'Wind_Speed': original_scale[:, 2],
        'Precipitation': original_scale[:, 3]
    }

    print("\nPredictions in original scale:")
    for day in range(len(original_scale)):
        print(f"\nDay {day + 1}:")
        print(f"Temperature: {original_scale[day, 0]:.1f}°F ({((original_scale[day, 0] - 32) * 5/9):.1f}°C)")
        print(f"Relative Humidity: {original_scale[day, 1]:.1f}%")
        print(f"Wind Speed: {original_scale[day, 2]:.1f}")
        print(f"Precipitation: {original_scale[day, 3]:.2f} mm")
    
    return original_scale, predictions_dict

predictions_reshaped = np.array([
    [0.30858016, 0.34533745, 0.06130841, -0.00353826],
    [0.2580503, 0.35065103, -0.00115667, 0.10565001],
    [0.3436694, 0.3117985, 0.08450161, -0.00145772],
    [0.2418874, 0.4070757, 0.05132692, 0.00118342],
    [0.24648532, 0.14871946, 0.03239831, 0.00098925]
])

original_predictions, predictions_by_feature = inverse_transform_weather_predictions(predictions_reshaped)


Predictions in original scale:

Day 1:
Temperature: 83.1°F (28.4°C)
Relative Humidity: 73.6%
Wind Speed: 5.6
Precipitation: 0.00 mm

Day 2:
Temperature: 82.6°F (28.1°C)
Relative Humidity: 73.8%
Wind Speed: 5.0
Precipitation: 2.64 mm

Day 3:
Temperature: 83.4°F (28.6°C)
Relative Humidity: 72.8%
Wind Speed: 5.8
Precipitation: 0.00 mm

Day 4:
Temperature: 82.4°F (28.0°C)
Relative Humidity: 75.2%
Wind Speed: 5.5
Precipitation: 0.03 mm

Day 5:
Temperature: 82.5°F (28.0°C)
Relative Humidity: 68.7%
Wind Speed: 5.3
Precipitation: 0.02 mm
